## Tokenizing with code

In [1]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4.1-mini")

tokens = encoding.encode("Hi my name is mitul and i love programming")

In [2]:
tokens

[12194, 922, 1308, 382, 2403, 361, 326, 575, 3047, 23238]

In [3]:
for token_id in tokens:
    token_text = encoding.decode([token_id])
    print(f"{token_id} = {token_text}")

12194 = Hi
922 =  my
1308 =  name
382 =  is
2403 =  mit
361 = ul
326 =  and
575 =  i
3047 =  love
23238 =  programming


In [4]:
encoding.decode([326])

' and'

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("AIza"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!") 

API key found and looks good so far!


In [12]:
from openai import OpenAI

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=api_key)

### A message to GEMINI is a list of dicts

In [15]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Mitul!"}
    ]

In [16]:
response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
response.choices[0].message.content

"Hi Mitul! It's nice to meet you.\n\nHow can I help you today?"

### OK let's now ask a follow-up question

In [17]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What's my name?"}
    ]

In [18]:
response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
response.choices[0].message.content

"As an AI, I don't have access to personal information like your name unless you tell me!\n\nSo, you'll have to tell me. 😊"

### Wait, wha??

We just told you!

What's going on??

Here's the thing: every call to an LLM is completely STATELESS. It's a totally new call, every single time. As AI engineers, it's OUR JOB to devise techniques to give the impression that the LLM has a "memory".

In [21]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Mitul!"},
    {"role": "assistant", "content": "Hi Mitul! How can I assist you today?"},
    {"role": "user", "content": "What's my name?"}
    ]

In [22]:
response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
response.choices[0].message.content

'Your name is Mitul!'

## To recap

With apologies if this is obvious to you - but it's still good to reinforce:

1. Every call to an LLM is stateless
2. We pass in the entire conversation so far in the input prompt, every time
3. This gives the illusion that the LLM has memory - it apparently keeps the context of the conversation
4. But this is a trick; it's a by-product of providing the entire conversation, every time
5. An LLM just predicts the most likely next tokens in the sequence; if that sequence contains "My name is Ed" and later "What's my name?" then it will predict.. Ed!

The ChatGPT product uses exactly this trick - every time you send a message, it's the entire conversation that gets passed in.

"Does that mean we have to pay extra each time for all the conversation so far"

For sure it does. And that's what we WANT. We want the LLM to predict the next tokens in the sequence, looking back on the entire conversation. We want that compute to happen, so we need to pay the electricity bill for it!
